In [1]:
import sys
import numpy as np
import pandas as pd
import os
import warnings

warnings.filterwarnings('ignore')

data_raw=pd.read_csv('/data/jiawenxiao/physionet0727/train_data.csv')
patient_list=data_raw.id.drop_duplicates().tolist()
N=len(patient_list)
X_t = np.empty([N], dtype=object)        # N * [t*d]
X_t_mask = np.empty([N], dtype=object)   # N * [t*d]
deltaT_t = np.empty([N], dtype=object)   # N * [t*d]
X_last_obsv= np.empty([N], dtype=object)
label=np.empty([N], dtype=object)

i=-1
for patient_id in  patient_list:   
    i+=1
    if i%1000==0:
        print('processing the {} records'.format(i))  
    data=data_raw[data_raw.id==patient_id] 
    
    X_last_obsv[i] = np.array(data.iloc[:,:-3].fillna(method='ffill').fillna(0))
       
    X_t[i] = np.array(data.iloc[:,:-3])
    X_t_mask[i] = 1-np.isnan(X_t[i]).astype('int8')
    X_t[i][np.isnan(X_t[i])] = 0
    deltaT_t[i] = np.zeros_like(X_t[i], dtype=int)
    deltaT_t[i][0,:] = 0
    for i_t in range(1, len(X_t[i])):
        deltaT_t[i][i_t,:] = 1 + (1-X_t_mask[i][i_t,:]) * deltaT_t[i][i_t-1,:]
    label[i]=np.array(data.iloc[:,-3]).reshape(-1,1)

processing the 0 records


KeyboardInterrupt: 

In [5]:
np.savez('/data/jiawenxiao/physionet0727/train_data.npz',X_t=X_t,X_last_obsv=X_last_obsv,
                                                        X_t_mask=X_t_mask,deltaT_t=deltaT_t,label=label)

In [6]:
filetype ='.psv'
test_path='/data/jiawenxiao/physionet0727/test_data/'

def get_filename(path,filetype):
    name =[]
    final_name = []
    for root,dirs,files in os.walk(path):
        for i in files:
            if filetype in i:
                name.append(i.replace(filetype,''))#生成不带‘.csv’后缀的文件名组成的列表
    final_name = [item +'.psv' for item in name]#生成‘.csv’后缀的文件名组成的列表
    return final_name

test_name=get_filename(test_path,filetype)

N=len(test_name)
X_t = np.empty([N], dtype=object)        # N * [t*d]
X_t_mask = np.empty([N], dtype=object)   # N * [t*d]
deltaT_t = np.empty([N], dtype=object)   # N * [t*d]
X_last_obsv= np.empty([N], dtype=object)
label=np.empty([N], dtype=object)


i=-1
for record_name in  test_name:
    i+=1
    if i%1000==0:
        print('processing the {} person'.format(i))  
        
    input_file = test_path+record_name    
    raw_data= pd.read_csv(input_file,sep='|')  
    data=raw_data.iloc[:,1:]
    
    num_rows = len(data)

    X_last_obsv[i] = np.array(data.iloc[:,:-3].fillna(method='ffill').fillna(0))
    X_t[i] = np.array(data.iloc[:,:-3])
    X_t_mask[i] = 1-np.isnan(X_t[i]).astype('int8')
    X_t[i][np.isnan(X_t[i])] = 0
    deltaT_t[i] = np.zeros_like(X_t[i], dtype=int)
    deltaT_t[i][0,:] = 0
    for i_t in range(1, len(X_t[i])):
        deltaT_t[i][i_t,:] = 1 + (1-X_t_mask[i][i_t,:]) * deltaT_t[i][i_t-1,:]
    label[i]=np.array(data.iloc[:,-3]).reshape(-1,1)
      

processing the 0 person
processing the 1000 person
processing the 2000 person
processing the 3000 person
processing the 4000 person
processing the 5000 person
processing the 6000 person
processing the 7000 person
processing the 8000 person


In [7]:
np.savez('/data/jiawenxiao/physionet0727/val_data.npz',X_t=X_t,X_last_obsv=X_last_obsv,
                                                                 X_t_mask=X_t_mask,deltaT_t=deltaT_t,label=label)